## Flow-based features

While using one packet at a time to detect intrusions is very simple and straightforward, one can argue that relevant context is lost: The model does not consider the position of the packet in a flow. Therefor, most RTF-based machine learning models in literature consider the packet flows rather than individual packets. Recall that a network flow is identified by its 5-tuple: Source address, destination address, source port, destination port and protocol. The main idea is to sort flows based on their 5-tuple (flow ID), and then extract features from the resulting flows. In the literature, this is usually done as follows:

1. Iterate over the dataset in chronological order
    1. For each packet, extract the flow ID (if applicable)
    2. Store the entire packet (or a truncated version) in a data structure containing all packets with that flow ID
2. Decide on the dimensions of your input features
3. Iterate over each flow
    1. Extract the input features
    2. Store the extracted features

These extracted, stored features can then be used for training and or model evaluation. Deciding on the input feature dimensions not has become slightly more complicated however, when compared to using individual packets. Mainly, there are two decisions that need to be made:

1. How many packets do you include for one sample;
2. Do you only consider the beginning of the flow, or the entire flow.

As an example, the model we will use in this exercise was trained on samples of 4x64 bytes, created by taking the first 64 bytes of 4 subsequent packets in a flow. All packets in such a flow were considered.

### In hardware

The above approach does not work in hardware, as it requires having access to the entire dataset at once, and being able to store that entire dataset at once while processing its data. When performing network intrusion detection in hardware, there is only a limited amount of storage and it is impossible to predict what future packets will arrive. Therefor, another approach is necessary. One solution is to sort incoming packets into buckets according to their flow ID, and to empty those buckets whenever there needs to be made space for new incoming packets. Emptying a bucket then amounts to creating an input sample for the detection model. (Image: *n* individual buckets, packets are stored according to their flow ID FID).

<img src="./images/32_sorting.gif" width="600" height="150" />

### In softeware: exercise
In software, both approaches are possible: Either sorting the entire dataset beforehand, or filling and emptying buckets based on the incoming traffic. In this exercise, we will sort the entire dataset dataset for extracting features and conducting inference. 

In [ ]:
import torch
import numpy as np
from lib.dataset import NIDSDataset


# Initialize the dataset
dset = NIDSDataset(
    packets_file="./data/dataset_packets_v2.npy", 
    labels_file="./data/dataset_labels_v1.npy")

# The flow_dict contains a list of packets for each flow ID:
# flow_dict = {
#    "id_1": [..., ...],
#    "id_2": [..., ...],
#    ...
# }
flow_dict = {}

# Also include the label in the flow ID to account for the possibility that different packets from the same flow 
# could be malicious.
# Iterate over the dataset, sort all valid input features to dictionary

for packet in dset:
    label = packet.get_label()
    
    # Own code here
    
    for word in packet:
        # Own code here
        pass
    
    
print("We extracted {} flow IDs.".format(len(flow_dict.keys())))

Once the packets have been sorted, you can use the dictionary to extract your actual features.
Tip: Initialize your input samples using *np.zeros(...)*, so that input samples that only have one, two or three instead of four packets available can account for the missing data. Similarly, in hardware the missing bytes would have to be zero-padded.

In [ ]:
input_buffer = []
label_buffer = []

for flow_id, packet_list in flow_dict.items():
    # Your code here
    pass

print("We prepared {} input samples".format(len(input_buffer)))

# The items in the input_buffer should be 320-sized numpy arrays
# These are then transform to tensors
input_tensors = []

for input_sample in input_buffer:
    # Turn the Numpy array into a PyTorch tensor
    input_tensor = torch.from_numpy(input_sample)
    # Change input dimensionality
    input_tensor = input_tensor.view(1, 1, 4 * 64)
    
    input_tensors.append(input_tensor)

Now we repeat the process of loading the CNN, performance inference and interpreting the results:

In [ ]:
from lib.nn_model import ExampleCNN1D4x64

model = ExampleCNN1D4x64(13)

# Load the trained parameters
model.load_state_dict(torch.load("./data/cnn1d4x64.model", map_location=torch.device('cpu')))

# Set the Batch Normalization layers for inference
model.eval()

In [ ]:
from lib.nn_model import label_mapping

predictions = []

for input_tensor in input_tensors:
    output_tensor = model(input_tensor.float())
    
    _, predicted = torch.max(output_tensor, 1)
    predictions.append(predicted)

predictions = torch.stack(predictions, 0).numpy()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Transform the indices to their corresponding class label:
labelled_predictions = []
for prediction in predictions:
    labelled_predictions.append(label_mapping[prediction[0]])

# Choose output figure size
_, ax = plt.subplots(figsize=(20, 20))

# Calculate the confusion matrix
cm = confusion_matrix(label_buffer, labelled_predictions, labels=label_mapping)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=label_mapping)
disp.plot(ax=ax)
plt.show()

In [ ]:
from sklearn.metrics import classification_report

from sklearn.metrics import classification_report

print(classification_report(label_buffer, labelled_predictions, 
                            labels=label_mapping, 
                            target_names=label_mapping,
                            digits=4,
                            zero_division=0
                           ))

We expect the following results:
- BENIGN F1 = 0.9421
- Bot F1 = 0.8235
- PortScan = 0.4684
- DDoS = 0.5741
- F1_weighted_avg = 0.7353

Now, clearly the generated samples were significantly more poorly classified using this second model, when compared to the model with simple input features. However, it should be noted that this is only a very limited representation of the the model performance, mainly due to the limited dataset size.

For a more fair comparison, we will briefly provide the full performance details for both models here:
Both models were trained on the entire CICIDS2017 dataset, specifically for the purpose of this workshop. We first sorted the entire dataset in flows, before extracting labelled samples. The extracted data was split in a training, validation and test set, consisting of respectively 75%, 15% and 10% of the dataset. We trained both models for 25 epochs at a learning rate of 0.0001 and a minibatch-size of 64. For the 1x64-model, each packet in a labelled flow constitutes an indivual sample, while the 4x64-model uses groups of 4 subsequent packets. Missing packets or packet bytes were zero-padded. For this configuration, we obtained the following results:

For *ExampleCNN1D1x64*:
```
                  precision    recall       fpr  f1-score       mcc   support
           BENIGN     99.99     99.82      0.09     99.91     98.81   1036764
              Bot     99.18     77.74      0.00     87.16     87.80       310
         PortScan    100.00     77.01      0.00     87.01     87.76        87
             DDoS     99.81    100.00      0.00     99.90     99.90     10973
       Web Attack     95.45     99.46      0.00     97.42     97.44       929
     Infiltration    100.00    100.00      0.00    100.00    100.00       598
    DoS GoldenEye     94.03     96.78      0.01     95.38     95.38      2605
         DoS Hulk     97.20     99.83      0.18     98.50     98.41     64536
 DoS Slowhttptest     92.57     93.21      0.01     92.89     92.88      1163
    DoS slowloris     97.62     96.03      0.00     96.82     96.82      1412
       Heartbleed    100.00    100.00      0.00    100.00    100.00       417
      FTP-Patator     99.53    100.00      0.00     99.77     99.77      2568
      SSH-Patator    100.00     99.96      0.00     99.98     99.98      2357
         accuracy                                             99.80   1124719
        macro avg     98.11     95.37      0.02     96.52     96.54   1124719
        micro avg     99.80     99.80      0.02     99.80     99.78   1124719
     weighted avg     99.80     99.80      0.09     99.80     98.78   1124719
```

For *ExampleCNN1D4x64*:

```
                  precision    recall       fpr  f1-score       mcc   support
           BENIGN     99.99     99.82      0.09     99.91     98.84   1036764
              Bot     96.93     81.61      0.00     88.62     88.94       310
         PortScan     97.40     86.21      0.00     91.46     91.63        87
             DDoS     99.86    100.00      0.00     99.93     99.93     10973
       Web Attack     98.94    100.00      0.00     99.46     99.47       929
     Infiltration    100.00    100.00      0.00    100.00    100.00       598
    DoS GoldenEye     97.39     95.97      0.01     96.67     96.67      2605
         DoS Hulk     97.20     99.99      0.18     98.57     98.49     64536
 DoS Slowhttptest     96.86     95.61      0.00     96.24     96.23      1163
    DoS slowloris     98.57     97.31      0.00     97.93     97.93      1412
       Heartbleed    100.00    100.00      0.00    100.00    100.00       417
      FTP-Patator     99.69     99.81      0.00     99.75     99.75      2568
      SSH-Patator    100.00     99.96      0.00     99.98     99.98      2357
         accuracy                                             99.81   1124719
        macro avg     98.68     96.64      0.02     97.58     97.53   1124719
        micro avg     99.81     99.81      0.02     99.81     99.80   1124719
     weighted avg     99.82     99.81      0.09     99.81     98.82   1124719
```
Note: *mcc* is the Matthews Correlation Coefficient, *fpr* is the False Positive Rate.

Both models achieve good results, with the *ExampleCNN1D4x64* model actually slightly surpassing its counterpart in both weighted and macro f1-score.